# Notebook: Thu thập dữ liệu đơn hàng từ Pancake POS API

### 1.1. Import thư viện & Config
### Giải thích
- Import các thư viện cần thiết (`os`, `json`, `requests`, `pandas`, `sqlalchemy`, …).  
- Đọc các biến môi trường từ `.env` (API key, thông tin DB, shop_id).  
- Giúp bảo mật thông tin thay vì ghi trực tiếp trong code.  

In [9]:
import os
import json
import time
import random
import requests
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.dialects.mysql import LONGTEXT, VARCHAR
from sqlalchemy.types import BigInteger, DateTime
from dotenv import load_dotenv
from datetime import datetime

# Load biến môi trường
load_dotenv()

API_KEY   = os.getenv("API_KEY")
DB_USER   = os.getenv("DB_USER")
DB_PASS   = os.getenv("DB_PASS")
DB_HOST   = os.getenv("DB_HOST")
DB_PORT   = os.getenv("DB_PORT")
DB_BRONZE = os.getenv("DB_BRONZE")
SHOP_ID   = os.getenv("SHOP_ID")  # nhớ thêm SHOP_ID vào file .env


### 1.2. Kết nối tới database Bronze
### Giải thích
- Sử dụng `SQLAlchemy` để tạo engine kết nối MySQL.  
- Database đích: schema Bronze, nơi lưu dữ liệu thô (raw).  
- In ra thông báo khi kết nối thành công.  

In [10]:
# Tạo engine tới database Bronze
engine_bronze = create_engine(
    f"mysql+pymysql://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_BRONZE}"
)

print(f"✅ Kết nối thành công tới database Bronze: {DB_BRONZE}")


✅ Kết nối thành công tới database Bronze: winner_bronze


### 1.3. Hàm lấy một trang dữ liệu đơn hàng từ API
### Giải thích
- Hàm `get_orders_page`:
  - Gọi API `/orders` với `shop_id`, `page`, `page_size`.
  - Trả về: danh sách khách hàng (`data`) và tổng số trang (`total_pages`).  
  - Có cơ chế retry tối đa 3 lần nếu lỗi (timeout hoặc HTTP error).  

In [11]:
def get_orders_page(shop_id, page=1, page_size=1000, max_retries=3):
    """
    Gọi API Pancake /shops/{shop_id}/orders để lấy một trang.
    Có retry + delay khi lỗi.
    """
    url = f"https://pos.pages.fm/api/v1/shops/{shop_id}/orders"
    params = {"api_key": API_KEY, "page": page, "page_size": page_size}
    
    for attempt in range(1, max_retries + 1):
        try:
            resp = requests.get(url, params=params, timeout=20)
            if resp.status_code == 200:
                j = resp.json()
                data = j.get("data", [])
                total_pages = j.get("total_pages") or j.get("meta", {}).get("total_pages")
                return data, total_pages
            else:
                print(f"⚠️ Lỗi status {resp.status_code} tại page {page}, attempt {attempt}")
        except Exception as e:
            print(f"⚠️ Exception tại page {page}: {e}, attempt {attempt}")
        time.sleep(2 * attempt)  # exponential backoff
    return [], None


### 1.4. Hàm ghi dữ liệu batch vào MySQL
### Giải thích
- Hàm `insert_orders_batch`:
  - Nhận DataFrame `df_batch`.
  - Ghi dữ liệu vào bảng `orders_raw` trong schema Bronze.
  - Các cột gồm: `shop_id`, `order_id`, `raw_json`, `extracted_at`.  

In [12]:
def insert_orders_batch(df_batch):
    df_batch.to_sql(
        "orders_raw",
        con=engine_bronze,
        if_exists="append",
        index=False,
        dtype={
            "shop_id": BigInteger(),
            "order_id": VARCHAR(50),
            "raw_json": LONGTEXT(),
            "extracted_at": DateTime()
        }
    )


### 1.5. Hàm fetch & load nhiều trang đơn hàng
### Giải thích
- Hàm `fetch_and_load_orders`:
  - Chạy vòng lặp qua từng trang API.
  - Chuyển đổi dữ liệu về DataFrame.
  - Lưu batch vào MySQL bằng `insert_orders_batch`.
  - Có delay ngẫu nhiên (1-3s) để tránh bị rate-limit.  

In [13]:
def fetch_and_load_orders(shop_id, start_page=1, page_size=1000):
    page = start_page
    while True:
        orders, total_pages = get_orders_page(shop_id, page, page_size)
        if not orders:
            print("📦 Không còn dữ liệu tại page", page)
            break

        # Tạo DataFrame batch
        df_batch = pd.DataFrame([{
            "shop_id": shop_id,
            "order_id": order.get("id"),
            "raw_json": json.dumps(order, ensure_ascii=False),
            "extracted_at": datetime.now()
        } for order in orders])

        # Insert batch vào Bronze
        insert_orders_batch(df_batch)
        print(f"✅ Page {page}/{total_pages} - Loaded {len(orders)} orders")

        if total_pages and page >= total_pages:
            break

        # Ngủ ngẫu nhiên 0.5–1.5s để tránh bị chặn
        time.sleep(random.uniform(0.5, 1.5))
        page += 1


### 1.6. Chạy tiến trình thu thập dữ liệu
### Giải thích
- Gọi hàm `fetch_and_load_orders` để thu thập toàn bộ dữ liệu đơn hàng.
- Kết quả: bảng `orders_raw` trong schema Bronze chứa dữ liệu thô từ API.  


In [14]:
fetch_and_load_orders(SHOP_ID, start_page=1, page_size=1000)


✅ Page 1/41 - Loaded 1000 orders
✅ Page 2/41 - Loaded 1000 orders
✅ Page 3/41 - Loaded 1000 orders
✅ Page 4/41 - Loaded 1000 orders
✅ Page 5/41 - Loaded 1000 orders
✅ Page 6/41 - Loaded 1000 orders
✅ Page 7/41 - Loaded 1000 orders
⚠️ Exception tại page 8: HTTPSConnectionPool(host='pos.pages.fm', port=443): Read timed out. (read timeout=20), attempt 1
✅ Page 8/41 - Loaded 1000 orders
⚠️ Exception tại page 9: HTTPSConnectionPool(host='pos.pages.fm', port=443): Read timed out. (read timeout=20), attempt 1
✅ Page 9/41 - Loaded 1000 orders
⚠️ Exception tại page 10: HTTPSConnectionPool(host='pos.pages.fm', port=443): Read timed out. (read timeout=20), attempt 1
✅ Page 10/41 - Loaded 1000 orders
⚠️ Exception tại page 11: HTTPSConnectionPool(host='pos.pages.fm', port=443): Read timed out. (read timeout=20), attempt 1
✅ Page 11/41 - Loaded 1000 orders
⚠️ Exception tại page 12: HTTPSConnectionPool(host='pos.pages.fm', port=443): Read timed out. (read timeout=20), attempt 1
✅ Page 12/41 - Loaded 